In [1]:
import overpy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

def geocode_address(address):
    geolocator = Nominatim(user_agent="your_app_name")
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)


def get_nearby_amenities(address):
    try:
        coordinates = geocode_address(address)

        overpass_query = f"""
            node(around:500, {coordinates[0]}, {coordinates[1]})
            ["amenity"];
            out;
        """

        api = overpy.Overpass()
        result = api.query(overpass_query)

        amenities = [node.tags.get('amenity', 'N/A') for node in result.nodes]
        return ", ".join(amenities)
    
    except Exception as e:
        return str(e)

def process_excel(input_file, output_file):
    df = pd.read_excel(input_file)

    df['Amenities'] = df['Address'].apply(get_nearby_amenities)

    df.to_csv(output_file, index=False)

# Example usage
input_excel_file = "final_input_addresses.xlsx"
output_csv_file = "final_output_amenities.csv"
process_excel(input_excel_file, output_csv_file)


In [2]:
# Create data frame
csv = pd.read_csv("final_output_amenities.csv", encoding='utf-8')
amt = pd.DataFrame(csv)

# Extracting all categories from amenities 
amenities_mentioned = amt["Amenities"].str.split().explode().unique().tolist()

# Print list of unique amenities
print(amenities_mentioned)

['post_box,', 'recycling,', 'fast_food,', 'restaurant,', 'parking,', 'pharmacy,', 'police,', 'pub,', 'bar,', 'cafe,', 'fire_station,', 'clinic,', 'bank,', 'bicycle_parking,', 'toilets,', 'lockers,', 'bench,', 'drinking_water,', 'bicycle_rental,', 'charging_station,', 'parking_entrance,', 'vending_machine,', 'piano,', 'doctors,', 'clock,', 'waste_basket,', 'boat_rental,', 'public_bookcase,', 'vending_machine', 'fuel,', 'motorcycle_parking,', 'bench', 'waste_disposal,', 'ice_cream,', 'studio,', 'community_centre,', 'place_of_worship,', 'bbq,', 'taxi,', 'fountain,', 'social_facility,', 'shelter,', 'arts_centre,', 'theatre,', 'dentist,', 'kindergarten,', 'school,', 'food_court,', 'atm,', 'parking_entrance', 'townhall,', 'healthcare,', 'social_centre,', 'dental_hygienist,', 'cinema,', 'fountain', 'bureau_de_change,', 'bus_station,', 'nightclub,', 'casino,', 'coworking_space,', 'accupuncture,', 'brothel,', 'bicycle_repair_station,', 'cannabis,', 'animal_shelter,', 'car_rental,', 'ferry_termi

In [3]:
excel = "3800_reviews_english.xlsx"

# Read Excel file into DataFrame
rating_amenities = pd.read_excel(excel)

# Drop specific columns from the original DataFrame
#keep place id, overall rating, individual rating
rating_amenities = rating_amenities.drop(["name", "review_text", "location_link", "reviews", "review_datetime_utc", "reviews_per_score_1", "reviews_per_score_2", "reviews_per_score_3", "reviews_per_score_4", "reviews_per_score_5", "query", "google_id", "reviews_link", "review_id", "review_pagination_id", "author_id", "review_text_english", "review_img_url", "review_img_url", "review_questions", "review_photo_ids", "owner_answer", "owner_answer_timestamp", "review_img_urls",  "owner_answer_timestamp_datetime_utc", "review_link", "review_timestamp", "review_likes", "reviews_id"], axis=1)

# Display new DataFrame
print(rating_amenities)


                         place_id  rating  review_rating
0     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              1
1     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              2
2     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              5
3     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              1
4     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              5
...                           ...     ...            ...
3830  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              4
3831  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5
3832  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5
3833  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              4
3834  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5

[3835 rows x 3 columns]


In [4]:
amenity_counts_dict = {amenity: [] for amenity in amenities_mentioned}

# Loop to count the amount of amenities per location
for amenities_str in amt["Amenities"]:
    for amenity in amenities_mentioned:
        count = amenities_str.count(amenity)
        
        amenity_counts_dict[amenity].append(count)

# Add the counts as new columns in amt
for amenity, counts in amenity_counts_dict.items():
    amt[f'{amenity}_counts'] = counts

amt["total_counts"] = amt[[f"{amenity}_counts" for amenity in amenities_mentioned]].sum(axis=1)

# Display
display(amt)



C:\Users\Usuario\AppData\Local\Temp\ipykernel_340932\2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[f'{amenity}_counts'] = counts
C:\Users\Usuario\AppData\Local\Temp\ipykernel_340932\2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[f'{amenity}_counts'] = counts
C:\Users\Usuario\AppData\Local\Temp\ipykernel_340932\2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

,Address,place_id,Amenities,"post_box,_counts","recycling,_counts","fast_food,_counts","restaurant,_counts","parking,_counts","pharmacy,_counts","police,_counts",...,parcel_locker_counts,bank_counts,'NoneType'_counts,object_counts,has_counts,no_counts,attribute_counts,'latitude'_counts,"nursing_home,_counts",total_counts
0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, pa...",4,17,4,8,10,1,1,...,0,2,0,0,0,1,0,0,0,215
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam",ChIJxZHiajgLxkcRlOo3zmIibOs,"parking, post_box, parking, fuel, charging_sta...",1,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,32
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam",ChIJYyOpnFQIxkcRErWFhYVuAeQ,"fuel, post_box, fast_food, pharmacy, pub, pub,...",2,16,5,6,3,1,0,...,0,0,0,0,0,0,0,0,0,204
3,"Spoorslag 29, 1082 MM Amsterdam",ChIJ15K9igQKxkcR8A7le6H02hI,"cafe, parking, place_of_worship, restaurant, c...",1,1,5,15,9,1,0,...,0,3,0,0,0,0,0,0,0,252
4,"Julianaplein Oost, 1097 DN Amsterdam",ChIJPd2JhH8JxkcRQ8jSHUchci0,"post_box, recycling, fast_food, restaurant, re...",4,15,4,8,9,0,1,...,0,2,0,0,0,1,0,0,0,209
5,"Termini 23, Amsterdam",ChIJn0fdmPwJxkcRYe7a-6JHfpI,"townhall, healthcare, social_centre, dentist, ...",1,33,5,5,3,0,0,...,0,0,0,0,0,0,0,0,0,239
6,"Stationsplein 39, 1012 AB Amsterdam",ChIJf_z8xbcJxkcRCc-nQwIMAP8,"restaurant, pub, pub, pub, restaurant, bureau_...",1,6,59,120,20,1,0,...,0,0,0,0,0,3,0,0,0,709
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam",ChIJ71V5XiPixUcRSKlH5OYy-iE,"recycling, recycling, recycling, post_box, res...",3,29,5,3,5,1,0,...,0,0,0,0,0,0,0,0,0,167
8,"Europaboulevard 4A, 1083 AD Amsterdam",ChIJLWMViCIKxkcRi1ThzDUCzCo,"theatre, conference_centre, parking, parking, ...",1,1,0,3,5,0,0,...,0,0,0,0,0,0,0,0,0,163
9,"Kattenburgerstraat 5, 1018 JA Amsterdam",ChIJ8exiz50JxkcRA0dHjOHV59g,"pub, post_box, restaurant, restaurant, restaur...",3,5,2,18,21,1,0,...,0,0,0,0,0,0,0,0,0,272


In [5]:
#add place ID
place_ID_df = pd.read_excel("final_input_addresses.xlsx")

# Merge DataFrames based on adress and safe place_id
merged_df = pd.merge(amt, place_ID_df, on = "Address", how = "left")

amt["place_id"] = place_ID_df.groupby("Address")["place_id"].transform("first")
display(amt)

,Address,place_id,Amenities,"post_box,_counts","recycling,_counts","fast_food,_counts","restaurant,_counts","parking,_counts","pharmacy,_counts","police,_counts",...,parcel_locker_counts,bank_counts,'NoneType'_counts,object_counts,has_counts,no_counts,attribute_counts,'latitude'_counts,"nursing_home,_counts",total_counts
0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, pa...",4,17,4,8,10,1,1,...,0,2,0,0,0,1,0,0,0,215
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam",ChIJxZHiajgLxkcRlOo3zmIibOs,"parking, post_box, parking, fuel, charging_sta...",1,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,32
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam",ChIJYyOpnFQIxkcRErWFhYVuAeQ,"fuel, post_box, fast_food, pharmacy, pub, pub,...",2,16,5,6,3,1,0,...,0,0,0,0,0,0,0,0,0,204
3,"Spoorslag 29, 1082 MM Amsterdam",ChIJ15K9igQKxkcR8A7le6H02hI,"cafe, parking, place_of_worship, restaurant, c...",1,1,5,15,9,1,0,...,0,3,0,0,0,0,0,0,0,252
4,"Julianaplein Oost, 1097 DN Amsterdam",ChIJPd2JhH8JxkcRQ8jSHUchci0,"post_box, recycling, fast_food, restaurant, re...",4,15,4,8,9,0,1,...,0,2,0,0,0,1,0,0,0,209
5,"Termini 23, Amsterdam",ChIJn0fdmPwJxkcRYe7a-6JHfpI,"townhall, healthcare, social_centre, dentist, ...",1,33,5,5,3,0,0,...,0,0,0,0,0,0,0,0,0,239
6,"Stationsplein 39, 1012 AB Amsterdam",ChIJf_z8xbcJxkcRCc-nQwIMAP8,"restaurant, pub, pub, pub, restaurant, bureau_...",1,6,59,120,20,1,0,...,0,0,0,0,0,3,0,0,0,709
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam",ChIJ71V5XiPixUcRSKlH5OYy-iE,"recycling, recycling, recycling, post_box, res...",3,29,5,3,5,1,0,...,0,0,0,0,0,0,0,0,0,167
8,"Europaboulevard 4A, 1083 AD Amsterdam",ChIJLWMViCIKxkcRi1ThzDUCzCo,"theatre, conference_centre, parking, parking, ...",1,1,0,3,5,0,0,...,0,0,0,0,0,0,0,0,0,163
9,"Kattenburgerstraat 5, 1018 JA Amsterdam",ChIJ8exiz50JxkcRA0dHjOHV59g,"pub, post_box, restaurant, restaurant, restaur...",3,5,2,18,21,1,0,...,0,0,0,0,0,0,0,0,0,272


In [6]:
#merge the two to have ratings and amenities in one DataFrame
rating_and_count = pd.merge(amt, rating_amenities, on = "place_id", how = "left")

#turn index column into 
rating_and_count.reset_index(inplace = True)

#Display merged DataFrame
display(rating_and_count)

#Save to csv
rating_and_count.to_csv("rating_and_count.csv", index = False)

all_columns = rating_and_count.columns.tolist()
print("List of all columns in the DataFrame:")
print(all_columns)


,index,Address,place_id,Amenities,"post_box,_counts","recycling,_counts","fast_food,_counts","restaurant,_counts","parking,_counts","pharmacy,_counts",...,'NoneType'_counts,object_counts,has_counts,no_counts,attribute_counts,'latitude'_counts,"nursing_home,_counts",total_counts,rating,review_rating
0,0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, pa...",4,17,4,8,10,1,...,0,0,0,1,0,0,0,215,3.2,1.0
1,1,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, pa...",4,17,4,8,10,1,...,0,0,0,1,0,0,0,215,3.2,2.0
2,2,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, pa...",4,17,4,8,10,1,...,0,0,0,1,0,0,0,215,3.2,5.0
3,3,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, pa...",4,17,4,8,10,1,...,0,0,0,1,0,0,0,215,3.2,1.0
4,4,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, pa...",4,17,4,8,10,1,...,0,0,0,1,0,0,0,215,3.2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3839,3839,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",1,0,20,15,3,1,...,0,0,0,1,0,0,0,149,4.0,4.0
3840,3840,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",1,0,20,15,3,1,...,0,0,0,1,0,0,0,149,4.0,5.0
3841,3841,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",1,0,20,15,3,1,...,0,0,0,1,0,0,0,149,4.0,5.0
3842,3842,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",1,0,20,15,3,1,...,0,0,0,1,0,0,0,149,4.0,4.0


List of all columns in the DataFrame:
['index', 'Address', 'place_id', 'Amenities', 'post_box,_counts', 'recycling,_counts', 'fast_food,_counts', 'restaurant,_counts', 'parking,_counts', 'pharmacy,_counts', 'police,_counts', 'pub,_counts', 'bar,_counts', 'cafe,_counts', 'fire_station,_counts', 'clinic,_counts', 'bank,_counts', 'bicycle_parking,_counts', 'toilets,_counts', 'lockers,_counts', 'bench,_counts', 'drinking_water,_counts', 'bicycle_rental,_counts', 'charging_station,_counts', 'parking_entrance,_counts', 'vending_machine,_counts', 'piano,_counts', 'doctors,_counts', 'clock,_counts', 'waste_basket,_counts', 'boat_rental,_counts', 'public_bookcase,_counts', 'vending_machine_counts', 'fuel,_counts', 'motorcycle_parking,_counts', 'bench_counts', 'waste_disposal,_counts', 'ice_cream,_counts', 'studio,_counts', 'community_centre,_counts', 'place_of_worship,_counts', 'bbq,_counts', 'taxi,_counts', 'fountain,_counts', 'social_facility,_counts', 'shelter,_counts', 'arts_centre,_counts'

In [41]:
rating_and_count.head()
rating_and_count.info()
print(rating_and_count.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3844 entries, 0 to 3843
Columns: 133 entries, index to review_rating
dtypes: float64(2), int64(128), object(3)
memory usage: 3.9+ MB
index                     int64
Address                  object
place_id                 object
Amenities                object
post_box,_counts          int64
                         ...   
'latitude'_counts         int64
nursing_home,_counts      int64
total_counts              int64
rating                  float64
review_rating           float64
Length: 133, dtype: object


In [52]:
# Correlation between features and ratings
# List of features for correlation analysis
selected_features = all_columns

# Ensure unique elements in the list
selected_features = list(set(selected_features))

# Calculate correlation matrix for selected features
correlation_matrix_selected = rating_and_count[selected_features + ['rating']].corr(numeric_only=True)
# Display the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix_selected)

# Find features with the highest correlation with ratings

"""
HERE IS THE PROBLEM: 
Tried to run without "by" parameter, which is not necessary here (technically)
It is not supposed to show any error! Super weird
"""


highest_correlation_selected = correlation_matrix_selected[['rating']].abs().sort_values(by="rating",ascending=False) 






print("\nFeatures with the highest correlation with ratings:")
print(highest_correlation_selected)

# Print the first 100 highest correlations with ratings. There are past 700 rows, so it should follow the general rule of thumb
# If we only use 100 columns
print("\nTop 100 Features with the Highest Correlation with Ratings:")
print(highest_correlation_selected.head(100))

Correlation Matrix:
                          car_sharing,_counts  fire_station,_counts  \
car_sharing,_counts                  1.000000             -0.050744   
fire_station,_counts                -0.050744              1.000000   
clock,_counts                       -0.053949              0.813973   
pharmacy,_counts                    -0.118526             -0.132343   
vending_machine,_counts              0.090933             -0.125366   
...                                       ...                   ...   
pedalo,_counts                      -0.033355             -0.056650   
no_counts                           -0.142255              0.015889   
community_centre,_counts            -0.039843             -0.067669   
college,_counts                      0.042963             -0.009172   
rating                               0.186433             -0.369076   

                          clock,_counts  pharmacy,_counts  \
car_sharing,_counts           -0.053949         -0.118526   
fire_

ValueError: The column label 'rating' is not unique.

In [50]:
"""
WORK IN PROGRESS: 

Make correlation score between all of them (all possible combinations). 
When highly correlated, drop one of the two. 
Take code from Vanessa?  

Matrix: all possible features x all possible features and correlation scores for all of them

->

import itertools
import pandas as pd

# Assuming 'Amenities' column contains strings with comma-separated amenity names
rating_and_count['Amenities'] = rating_and_count['Amenities'].astype(str)
rating_and_count['Amenities'] = rating_and_count['Amenities'].str.split(',')

# Create a list of all amenities
amenities = list(set(itertools.chain.from_iterable(rating_and_count['Amenities'])))

# Create all possible combinations of amenities
combinations = list(itertools.combinations(amenities, 2))

# Create an empty DataFrame to store correlation scores
correlation_scores = pd.DataFrame(index=amenities, columns=amenities)

# Calculate correlation scores for all combinations
for amenity1, amenity2 in combinations:
    correlation_score = rating_and_count[amenity1].corr(rating_and_count[amenity2])
    correlation_scores.loc[amenity1, amenity2] = correlation_score
    correlation_scores.loc[amenity2, amenity1] = correlation_score

# Identify highly correlated features
highly_correlated_features = set()
for amenity in amenities:
    correlated_with_amenity = correlation_scores[amenity][abs(correlation_scores[amenity]) > 0.8].index.tolist()
    highly_correlated_features.update(correlated_with_amenity)

# Display highly correlated features
print("\nHighly Correlated Features to be Dropped:")
print(highly_correlated_features)

# Drop the highly correlated features from the dataframe
rating_and_count_filtered = rating_and_count.drop(columns=highly_correlated_features)

# Display the filtered dataframe
print("\nFiltered DataFrame after dropping highly correlated features:")
print(rating_and_count_filtered)


ORIGINAL WORKING CODE BELLOW: 
"""
# Calculate correlation matrix for all columns
correlation_matrix = rating_and_count.corr(numeric_only=True)

# Identify highly correlated features
highly_correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            highly_correlated_features.add(colname)

# Display highly correlated features
print("\nHighly Correlated Features to be Dropped:")
print(highly_correlated_features)

# Drop the highly correlated features from the dataframe
rating_and_count_filtered = rating_and_count.drop(columns=highly_correlated_features)

# Display the filtered dataframe
print("\nFiltered DataFrame after dropping highly correlated features:")
print(rating_and_count_filtered)

#Drops 65 columns, 68 amenities remaining



Highly Correlated Features to be Dropped:
{'has_counts', 'theatre,_counts', 'drinking_water_counts', 'K+R,_counts', 'clock,_counts', 'bank_counts', 'karaoke_box,_counts', 'nightclub,_counts', 'atm,_counts', 'vending_machine,_counts', 'toilets,_counts', 'lockers,_counts', 'bank,_counts', 'pub,_counts', 'bar,_counts', 'parking_entrance_counts', 'bicycle_repair_station,_counts', 'recycling_counts', 'mailboxes,_counts', 'photo_booth,_counts', 'restaurant_counts', 'station,_counts', 'office_units,_counts', 'restaurant,_counts', 'luggage_locker,_counts', 'events_venue,_counts', 'total_counts', 'cannabis,_counts', 'cooking_school,_counts', 'attribute_counts', 'dental_hygienist,_counts', 'accupuncture,_counts', 'post_office,_counts', 'car_wash,_counts', 'music_studio,_counts', 'ticket_validator,_counts', 'bench_counts', 'social_facility,_counts', 'waste_basket,_counts', 'warehouse,_counts', 'bicycle_parking,_counts', "'latitude'_counts", 'vending_machine_counts', 'taxi,_counts', 'conference_c